In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
import pandas as pd
import scipy.stats

In [4]:
# df.groupby(["is_arrested","driver_gender"])["id"].count()

In [5]:
def chi2testcols(df,col1,col2):
    contingency = pd.crosstab(df[col1],df[col2])
    return scipy.stats.chi2_contingency(contingency,correction=True)

In [6]:
df = pd.read_csv("../data/police_stop/CT_police_stop.csv",index_col=0)

In [7]:
col_of_interest = "is_arrested"
for col in df.columns:
    if col!="id" and col!=col_of_interest:
        chi2,p,dof,expected = chi2testcols(df,col_of_interest,col)
        print col,chi2,p, dof

state 0.0 1.0 0
driver_gender 361.582454182 1.27353587902e-80 1
driver_race 431.010885323 4.23997940993e-93 3
search_conducted 15083.5763734 0.0 1
contraband_found 7167.29901508 0.0 1
stop_outcome 112380.690612 0.0 2
stop_duration 37038.0496383 0.0 2
stop_time_of_day 1316.13516252 4.64302100104e-285 3
driver_age_category 187.024993528 1.68909660711e-38 5
speeding_violations 839.862225001 1.16293522996e-184 1
other_violations 1375.13629089 5.31374175281e-301 1
registration_plates_violations 239.845797738 4.24940911363e-54 1
moving_violation 955.414082552 8.82792981365e-210 1
cell_phone_violations 265.617007322 1.02374525649e-59 1


In [8]:
df = pd.read_csv("../data/mushrooms.csv")

In [9]:
col_of_interest = "cap_surface"
for col in df.columns:
    if col!="id" and col!=col_of_interest:
        chi2,p,dof,expected = chi2testcols(df,col_of_interest,col)
        print col,chi2,p

type 315.042831208 5.51842703865e-68
cap_shape 1011.49300341 4.63577768747e-206
cap_color 1236.82895751 1.23799754405e-243
bruises 146.69581085 1.35984418687e-31
odor 1385.70849757 5.62429497299e-278
gill_attachment 361.145242221 5.75818576945e-78
gill_spacing 929.337925721 3.83113074891e-201
gill_size 625.211042055 3.44979719064e-135
gill_color 1617.4942974 0.0
stalk_shape 50.4072499459 6.5428077197e-11
stalk_root 2412.97673157 0.0
stalk_surface_above_ring 506.230073969 2.66396008634e-103
stalk_surface_below_ring 699.33302946 9.62195736733e-145
stalk_color_above_ring 1553.5284428 0.0
stalk_color_below_ring 1615.40020144 0.0
veil_type 0.0 1.0
veil_color 440.046874302 3.84560188068e-89
ring_number 159.719314803 6.79034249873e-32
ring_type 1230.82739978 3.97573732498e-256
spore_print_color 1318.85162183 1.07639844485e-263
population 1674.0191573 0.0
habitat 1535.54716914 0.0


In [10]:
col_of_interest = "type"
for col in df.columns:
    if col!="id" and col!=col_of_interest:
        chi2,p,dof,expected = chi2testcols(df,col_of_interest,col)
        print col,chi2,p

cap_shape 489.919953619 1.19645656859e-103
cap_surface 315.042831208 5.51842703865e-68
cap_color 387.597768977 6.05581459834e-78
bruises 2041.41564746 0.0
odor 7659.72674017 0.0
gill_attachment 133.986181287 5.50170741186e-31
gill_spacing 984.143333014 5.02297761373e-216
gill_size 2366.83425691 0.0
gill_color 3765.71408624 0.0
stalk_shape 84.1420382655 4.60474621216e-20
stalk_root 1344.44052655 7.70204790494e-290
stalk_surface_above_ring 2808.28628742 0.0
stalk_surface_below_ring 2684.47407633 0.0
stalk_color_above_ring 2237.89849645 0.0
stalk_color_below_ring 2152.3908906 0.0
veil_type 0.0 1.0
veil_color 191.223701525 3.32097274917e-41
ring_number 374.736830827 4.23575764172e-82
ring_type 2956.61927806 0.0
spore_print_color 4602.03317008 0.0
population 1929.7408909 0.0
habitat 1573.77726083 0.0


In [11]:
df = pd.read_csv("../data/titanic.csv")

In [12]:
col_of_interest = "survived"
for col in df.columns:
    if col!="id" and col!=col_of_interest:
        chi2,p,dof,expected = chi2testcols(df,col_of_interest,col)
        print col,chi2,p

name 1310.78039269 0.480934328391
pc_class 172.299118274 3.85231550242e-38
age 100.032249616 0.0236375211674
sex 329.841992465 1.04040309197e-73
sexcode 329.841992465 1.04040309197e-73


In [13]:
df = pd.read_csv("../data/turn_cheetah_query_log_for_uiuc_v1.csv")

In [15]:
df.head()

,job_id,user_company_id,beneficiary_company_id,window_start_time,window_end_time,submission_time,start_time,finish_time,is_successful,is_multi_query,...,hdfs_bytes_written,total_launched_maps,total_launched_reduces,map_input_records,map_output_records,reduce_input_records,reduce_input_groups,reduce_output_records,slots_millis_maps,slots_millis_reduces
0,job-3432142755,company-2342552567,company-1554488253,1998_01_15 05:00:00,1998_04_04 04:59:59,1998_04_04 04:55:39,1998_04_04 07:19:20,1998_04_04 07:20:42,1,0,...,587309,1621,32,68201013,11281,8649,4367,4367,15713603,646243
1,job-3698996308,company-2342552567,company-1554488253,1998_03_27 05:00:00,1998_04_04 04:59:59,1998_04_04 04:56:16,1998_04_04 07:22:21,1998_04_04 07:24:51,1,0,...,25056,3960,9,883757542,198710,198710,457,457,48298333,190295
2,job-280950780,company-2342552567,company-1662668658,1998_02_08 05:00:00,1998_04_04 04:59:59,1998_04_04 05:41:10,1998_04_04 07:43:57,1998_04_04 07:45:11,1,0,...,11073633,1453,31,11809839,135028,79339,34138,34138,13158307,559258
3,job-909565294,company-2342552567,company-186432602,1997_10_07 04:00:00,1998_04_03 04:59:59,1998_04_04 05:41:20,1998_04_04 18:35:29,1998_04_04 18:58:45,1,0,...,997,18956,10,11634594931,8280,8280,2,2,239394231,1062991
4,job-246208440,company-2342552567,company-186432602,1998_03_01 05:00:00,1998_04_01 04:59:59,1998_04_04 05:41:26,1998_04_04 18:36:05,1998_04_04 18:46:50,1,0,...,997,4147,10,6980591912,8280,8280,2,2,107177437,292714


In [16]:
# col_of_interest = "slots_millis_reduces"
# for col in df.columns:
#     if "id" in col and col!=col_of_interest:
#         chi2,p,dof,expected = chi2testcols(df,col_of_interest,col)
#         print col,chi2,p